In [1]:
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import os
import pandas as pd
import emcee

%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [52]:
DATALC_PATH = "/Users/mariavincenzi/PhD/pycoco_2/prepare_photometry/Final_LC_NOMW/"
DATAMAIN_PATH= "/Users/mariavincenzi/PhD/pycoco_2/data/"

CSP_SNe = ['SN2004fe', 'SN2005bf', 'SN2006V', 'SN2007C', 'SN2007Y',
           'SN2009bb',  'SN2008aq', 'SN2006T', 'SN2004gq', 'SN2004gt',
           'SN2004gv','SN2006ep', 'SN2008fq', 'SN2006aa']

Vega_filt = ['Bessell_U', 'Bessell_B','Bessell_V','Bessell_R','Bessell_I', 
            'swift_UVW2', 'swift_UVM2', 'swift_UVW1', 'swift_U', 'swift_B','swift_V',
            'Y', 'H', 'J', 'K', 'Ks']
AB_filt = ['sdss_u', 'sdss_g','sdss_r','sdss_i','sdss_z', 
            "sdss_u'", "sdss_g'", "sdss_r'", "sdss_i'", "sdss_z'"]

exclude_filt = ['H', 'J', 'K', 'Ks','KS', 'Y']
info_objects = pd.read_csv(DATAMAIN_PATH+'info/FINAL_info.dat')

In [173]:

class SNPhotometryClass():
    """Class with photometry for each object:
            - load the photometry from the DATA folder
            - get the phootmetry in each filter
            - plot the raw photometry 
            - fit the photometry using GP
    """
    
    def __init__(self, main_path, lc_path, snname, verbose=False):
        """
        """

        ## Initialise the class variables
        self.main_path = main_path
        self.lc_data_path = lc_path+'/'
        #self.lc_data_path = '/Users/mariavincenzi/PhD/pycoco_2/data/lc/'

        self.spec_data_path = main_path+'/spec/'
        self.snname = snname   
        self.set_data_directory(verbose)

    def set_data_directory(self, verbose):
        """
        Set a new data directory path.
        Enables the data directory to be changed by the user.
        """
        SNphotometry_PATH = os.path.join(self.lc_data_path, '%s.dat'%self.snname)
        
        try:
            if verbose: print('Looking for Photometry for %s in%s'%(self.snname, SNphotometry_PATH))
            if os.path.isfile(SNphotometry_PATH):
                if verbose: print ('Got it!')
                self.sn_rawphot_file = SNphotometry_PATH
                pass
            else:
                if not os.path.isdir(self.lc_data_path):
                    print ('I cant find the directory with photometry. Check %s'%self.lc_data_path)
                    pass
                else: 
                    print ('I cant find the file with photometry. Check %s'%SNphotometry_PATH)
                    pass
    
        except Exception as e:
            print (e)

    def load(self, verbose = False):
        """
        Loads a single photometry file.
        with ('MJD', 'flux', 'flux_err', 'filter')
        
        Parameters
        - verbose
        ----------
        Returns
        - photometry in all filters
        -------
        """
        if verbose: print('Loading %s'%self.sn_rawphot_file)
        try:
            lc_file = np.genfromtxt(self.sn_rawphot_file,
                                    names=['MJD','filter','flux','flux_err','FilterSet','Source'], 
                                    usecols=[0,1,2,3,4,5], 
                                    dtype=None,encoding="utf-8")

            mask_filt = np.array([f not in exclude_filt for f in lc_file['filter']])
            lc_no_badfilters = lc_file[mask_filt]
            mask_filt = np.array([~np.isnan(f) for f in lc_no_badfilters['flux']])
            self.phot = lc_no_badfilters[mask_filt]
            self.avail_filters = np.unique(self.phot['filter'])
            if verbose: print ('Photometry loaded')

        except Exception as e:
            print (e)
            print ('Are you sure you gave me the right format? Check documentation in case.')

    def get_availfilter(self, verbose = False):
        """
        get available filter for this SN
        """
        #if photometry is not already loaded, load it!
        if (not hasattr(self, "phot"))|(not hasattr(self, "avail_filters")):
            self.load()
        return self.avail_filters
        
    def get_singlefilter(self, single_filter, verbose = False):
        """
        Loads from photometry file just 1 filter photometry.
        with ('MJD', 'flux', 'flux_err', 'filter')
        
        Parameters
        - verbose
        ----------
        Returns
        - photometry in all filters
        -------
        """
        #if photometry is not already loaded, load it!
        if not hasattr(self, "phot"):
            self.load()

        if not (isinstance(single_filter, str)):
            print ('Single filter string please')
            return None
        
        if single_filter not in self.avail_filters:
            if verbose: print ('Looks like the filter you are looking for is not available')
            return None
        
        filt_index = self.phot['filter']==single_filter
        return self.phot[filt_index]
        
    def corr_dust_singlefilter(self, filter_name):
        if not hasattr(self, "corr_factors"):
            self.corr_factors = {}
        corr_factors_dict = self.corr_factors            

        self.get_dust()
        self.get_redshift()

        if 'swift' in filter_name:
            w,t = w,t=wtf.loadFilter(FILTER_PATH+'/Swift/%s.dat'%filter_name)
        elif self.snname in CSP_SNe:
            w,t = w,t=wtf.loadFilter(FILTER_PATH+'/Site3_CSP/%s.txt'%filter_name)
        else:
            w,t = w,t=wtf.loadFilter(FILTER_PATH+'/GeneralFilters/%s.dat'%filter_name)
            
        w_SNframe = w/(1.+self.redshift)
        
        if filter_name in Vega_filt: band = wtf.Band_Vega(w_SNframe,t)
        elif filter_name in AB_filt: band = wtf.Band_AB(w_SNframe,t)
        ext_corr_Host = (1./band.extinction(self.Hostebv, 'CCM')).value

        if filter_name in Vega_filt: band = wtf.Band_Vega(w,t)
        elif filter_name in AB_filt: band = wtf.Band_AB(w,t)
        ext_corr_MW = (1./band.extinction(self.MWebv, 'CCM')).value
        
        corr_factors_dict[filter_name] = ext_corr_Host*ext_corr_MW
        self.corr_factors = corr_factors_dict           

    
    def get_dust(self):
        if self.snname not in info_objects.Name.values:
            raise Exception('This SN is not in the FINAL_INFO.dat file')
        else:
            info_singleobj = info_objects[info_objects.Name==self.snname]
            self.MWebv = info_singleobj['MW ebv'].values[0]
            Host_ebv = info_singleobj['Host ebv'].values[0]
            if 'ddag' in Host_ebv: 
                Host_ebv = float(Host_ebv.replace('ddag',''))
            self.Hostebv = float(Host_ebv)
        
    def get_redshift(self):
        info_singleobj = info_objects[info_objects.Name==self.snname]
        self.redshift = info_singleobj['z'].values[0]

    def correct_final_LC(self, name_file = None):
        for ff in self.get_availfilter():
            self.corr_dust_singlefilter(ff)
        
        lc_file = pd.DataFrame(self.phot)
        corr_dust_array = [self.corr_factors[f] for f in lc_file['filter']]
        lc_file['flux_corr'] = corr_dust_array*lc_file['flux'].values
        lc_file['flux_corr_err'] = corr_dust_array*lc_file['flux_err'].values

        return lc_file

In [174]:
import sys
sys.path.insert(0, '/Users/mariavincenzi/PhD/Photometry_Utils/')

import what_the_flux as wtf

In [175]:
FILTER_PATH = "/Users/mariavincenzi/PhD/pycoco_2/prepare_photometry/Sullivan_Filters/"

w,t=wtf.loadFilter(FILTER_PATH+'GeneralFilters/Bessell_B.dat')
band = wtf.Band_Vega(w,t)
ext_corr = 1./band.extinction(0.1, 'CCM')
print (ext_corr)

1.4613061476437499


In [176]:
w_SNrestframe = w/(1.5)

band = wtf.Band_Vega(w_SNrestframe,t)
ext_corr = 1./band.extinction(0.1, 'CCM')
print (ext_corr)

1.712531477835318


In [177]:
import shutil
sn_list = [f.replace('.dat','') for f in os.listdir(DATALC_PATH) if '.dat' in f]

for sn in sn_list:
    try:
        prova2 = SNPhotometryClass(main_path=DATAMAIN_PATH, lc_path= DATALC_PATH,snname=sn, verbose=True)
        prova2.load()
        prova2.get_availfilter()
        df = prova2.correct_final_LC()
        df_to_print = df[['MJD', 'filter', 'flux_corr', 'flux_corr_err', 'FilterSet', 'Source']]
        df_to_print.to_csv(prova2.sn_rawphot_file.replace('/Final_LC_NOMW/','/Final_LC_NOMW_dust_corr/'), na_rep='nan',
              index=False, sep='\t', header=['#MJD','band','Flux','Flux_err','FilterSet', 'Source'])
    except Exception as e:
        print (e, sn, 'NO CORRECTION')

Looking for Photometry for ASASSN14jb in/Users/mariavincenzi/PhD/pycoco_2/prepare_photometry/Final_LC_NOMW//ASASSN14jb.dat
Got it!
Looking for Photometry for ASASSN15no in/Users/mariavincenzi/PhD/pycoco_2/prepare_photometry/Final_LC_NOMW//ASASSN15no.dat
Got it!
This SN is not in the FINAL_INFO.dat file ASASSN15no NO CORRECTION
Looking for Photometry for ASASSN15oz in/Users/mariavincenzi/PhD/pycoco_2/prepare_photometry/Final_LC_NOMW//ASASSN15oz.dat
Got it!
Looking for Photometry for iPTF13bvn in/Users/mariavincenzi/PhD/pycoco_2/prepare_photometry/Final_LC_NOMW//iPTF13bvn.dat
Got it!
Looking for Photometry for SN1993J in/Users/mariavincenzi/PhD/pycoco_2/prepare_photometry/Final_LC_NOMW//SN1993J.dat
Got it!
Looking for Photometry for SN1994I in/Users/mariavincenzi/PhD/pycoco_2/prepare_photometry/Final_LC_NOMW//SN1994I.dat
Got it!
Looking for Photometry for SN1996cb in/Users/mariavincenzi/PhD/pycoco_2/prepare_photometry/Final_LC_NOMW//SN1996cb.dat
Got it!
This SN is not in the FINAL_INFO.d

Looking for Photometry for SN2009ip in/Users/mariavincenzi/PhD/pycoco_2/prepare_photometry/Final_LC_NOMW//SN2009ip.dat
Got it!
Looking for Photometry for SN2009iz in/Users/mariavincenzi/PhD/pycoco_2/prepare_photometry/Final_LC_NOMW//SN2009iz.dat
Got it!
Looking for Photometry for SN2009jf in/Users/mariavincenzi/PhD/pycoco_2/prepare_photometry/Final_LC_NOMW//SN2009jf.dat
Got it!
Looking for Photometry for SN2009kr in/Users/mariavincenzi/PhD/pycoco_2/prepare_photometry/Final_LC_NOMW//SN2009kr.dat
Got it!
Looking for Photometry for SN2009mg in/Users/mariavincenzi/PhD/pycoco_2/prepare_photometry/Final_LC_NOMW//SN2009mg.dat
Got it!
This SN is not in the FINAL_INFO.dat file SN2009mg NO CORRECTION
Looking for Photometry for SN2009N in/Users/mariavincenzi/PhD/pycoco_2/prepare_photometry/Final_LC_NOMW//SN2009N.dat
Got it!
Looking for Photometry for SN2010al in/Users/mariavincenzi/PhD/pycoco_2/prepare_photometry/Final_LC_NOMW//SN2010al.dat
Got it!
Looking for Photometry for SN2010bq in/Users/mar